**RANDOM FORSET**

Let's begin by importing nessesary libraries

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

Let's load the data and divide it into training and test sets. Note that in case of random forest model there is no need to scale the data with the Standard Scaler

In [5]:
df = pd.read_csv('masterfile done.csv')

X = df.iloc[:, 1:-2]
y = df['future result']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify= y, random_state = 1)

Now let's run a simple random forest model and see how it performs

In [38]:
rf = RandomForestClassifier(random_state= 1)
rf.fit(X_train, y_train)
score = rf.score(X_test, y_test)
print("Random forest accuracy ", score)

Random forest accuracy  0.6643835616438356


Now let's introduce cross validation and param grid to find the best set of hyperparameters

In [39]:
kf = KFold(n_splits=10, shuffle=True, random_state=1)
param_grid = {'criterion': ['gini', 'entropy', 'log_loss'],
               'max_depth': range(1, 6),
               'n_estimators': [100, 300, 400]}

rf_cv = GridSearchCV(rf, param_grid, cv = kf)
rf_cv.fit(X_train, y_train)
print('rf accuracy', rf_cv.best_score_)
print('rf best params: ', rf_cv.best_params_)

rf accuracy 0.6631889183613322
rf best params:  {'criterion': 'entropy', 'max_depth': 3, 'n_estimators': 200}


It may seem a little counterintuitive that the model now performs a tiny bit worse, but that's thanks to the cross validation. Now let's try this for every possible combination of variables

In [42]:
def powerset(s):
   """funcion that spits out every subset of a given set"""
   all_sets = []
   x = len(s)
   for i in range(1 << x):
       #print([s[j] for j in range(x) if (i & (1 << j))])
       subsets = [s[j] for j in range(x) if (i & (1 << j))]
       all_sets.append(subsets)
   return(all_sets)
     
sets = powerset([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

In [46]:
for i in range(1, len(df)):
    X = df.iloc[:, sets[i]]
    y = df['future result']
    list = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1, stratify= y)
    kf = KFold(n_splits=10, shuffle=True, random_state=1)
    rf = RandomForestClassifier()
    param_grid = {'criterion': ['gini', 'entropy', 'log_loss'],
               'max_depth': range(1, 6),
               'n_estimators': [100, 300, 400]}
    rf_cv = GridSearchCV(rf, param_grid, cv = kf)
    rf_cv.fit(X_train, y_train)
    list.append([rf_cv.best_score_, rf_cv.best_params_])

df_r = pd.DataFrame(list)
df_r.to_csv('Random Forest Full Test.csv')
